Topic : Opening a new Bermese Restaurant in Toronto
In this notebook, I will be creating clusters to find the most suitable location to open an authentic Burmese restaurant in Toronto, Canada.

In [4]:
#importing necessary libraries
import requests
import pandas as pd

In [5]:
#scrapping neighborhoods in Canada
url  = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
if page.status_code == 200:
    print('Page download successful')
else:
    print('Page download error. Error code: {}'.format(page.status_code))

Page download successful


In [6]:
df_html = pd.read_html(url, header=0, na_values = ['Not assigned'])[0]
df_html.head()

,Postal Code,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [7]:

#Drop the the rows on which the Borough is empty
df_html.dropna(subset=['Borough'], inplace=True)

In [8]:
#Check Neighborhood is empty but Borough exists
n_empty_neighborhood = df_html[df_html['Neighbourhood'].isna()].shape[0]
print('Number of rows on which Neighborhood column is empty: {}'.format(n_empty_neighborhood))

KeyError: 'Neighbourhood'

In [9]:

#Show which neighborhood is emtpy but Borough exists
df_html[df_html['Neighbourhood'].isna()]

KeyError: 'Neighbourhood'

In [10]:

#Replace empty Neighborhood with Borough name and check again
df_html['Neighbourhood'].fillna(df_html['Borough'], inplace=True)
n_empty_neighborhood = df_html[df_html['Neighbourhood'].isna()].shape[0]
print('Number of rows on which Neighborhood column is empty: {}'.format(n_empty_neighborhood))

KeyError: 'Neighbourhood'

In [11]:
#Confirm that Queen's Park Neighborhood is not empty now:
df_html[df_html['Borough']=="Queen's Park"]

,Postal Code,Borough,Neighborhood


In [14]:
#Group by Postcode / Borough
df_postcodes = df_html.groupby(['Postcode','Borough']).Neighbourhood.agg([('Neighbourhood', ', '.join)])
df_postcodes.reset_index(inplace=True)
df_postcodes.head(5)

KeyError: 'Postcode'

In [15]:
#Drop the the rows on which the Borough is empty
df_html.dropna(subset=['Borough'], inplace=True)

In [16]:

#Check Neighborhood is empty but Borough exists
n_empty_neighborhood = df_html[df_html['Neighbourhood'].isna()].shape[0]
print('Number of rows on which Neighborhood column is empty: {}'.format(n_empty_neighborhood))

KeyError: 'Neighbourhood'

In [17]:

print('The shape of the dataset is:',df_postcodes.shape)

NameError: name 'df_postcodes' is not defined

In [18]:
#to make it easier, we will store this in csv format.
#Export to .CSV
df_postcodes.to_csv('Toronto_Postcodes.csv')

NameError: name 'df_postcodes' is not defined

In [19]:
import numpy as np

In [20]:
#Read CSV file from link and load into dataframe
url_csv = 'http://cocl.us/Geospatial_data'
df_coordinates = pd.read_csv(url_csv)
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:

#use the previously cleaned data
df_neighborhoods = pd.read_csv('Toronto_Postcodes.csv',index_col=[0])
df_neighborhoods.head()

FileNotFoundError: [Errno 2] File b'Toronto_Postcodes.csv' does not exist: b'Toronto_Postcodes.csv'

In [22]:
# Make sure both dataframes have the same 
df_coordinates.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_neighborhoods.rename(columns={'Postcode': 'PostalCode'}, inplace=True)

NameError: name 'df_neighborhoods' is not defined

In [23]:
# Merge both datasets
df_neighborhoods_coordinates = pd.merge(df_neighborhoods, df_coordinates, on='PostalCode')
df_neighborhoods_coordinates.head()

NameError: name 'df_neighborhoods' is not defined

In [25]:
# Check coordinates for a couple of neighborhoods
df_neighborhoods_coordinates[(df_neighborhoods_coordinates['PostalCode']=='M5G') |
                             (df_neighborhoods_coordinates['PostalCode']=='M2H') ]

NameError: name 'df_neighborhoods_coordinates' is not defined

In [26]:
#Export to .CSV
df_neighborhoods_coordinates.to_csv('Toronto_Postcodes_2.csv')

NameError: name 'df_neighborhoods_coordinates' is not defined

In [27]:
import folium
from sklearn.cluster import KMeans

In [28]:

# Read .csv file from above
df = pd.read_csv('Toronto_Postcodes_2.csv', index_col=0)
df.head()

FileNotFoundError: [Errno 2] File b'Toronto_Postcodes_2.csv' does not exist: b'Toronto_Postcodes_2.csv'

In [29]:

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

NameError: name 'df' is not defined

In [30]:
df.rename(columns={'Neighbourhood': 'Neighborhood'}, inplace=True)

NameError: name 'df' is not defined

In [31]:
#count Bourough and Neighborhood
df.groupby('Borough').count()['Neighborhood']

NameError: name 'df' is not defined

In [32]:
df_toronto = df[df['Borough'].str.contains('Toronto')]
df_toronto.reset_index(inplace=True)
df_toronto.drop('index', axis=1, inplace=True)
df_toronto.head()

NameError: name 'df' is not defined

In [33]:
#Check the number of neighborhoods
print(df_toronto.groupby('Borough').count()['Neighborhood'])

NameError: name 'df_toronto' is not defined

In [34]:
#Create list with the Boroughs (to be used later)
boroughs = df_toronto['Borough'].unique().tolist()

NameError: name 'df_toronto' is not defined

In [35]:
#Obtain the coordinates from the dataset itself, just averaging Latitude/Longitude of the current dataset 
lat_toronto = df_toronto['Latitude'].mean()
lon_toronto = df_toronto['Longitude'].mean()
print('The geographical coordinates of Toronto are {}, {}'.format(lat_toronto, lon_toronto))

NameError: name 'df_toronto' is not defined

In [36]:
borough_color = {}
for borough in boroughs:
    borough_color[borough]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3)) #Random color

NameError: name 'boroughs' is not defined

In [37]:
map_toronto = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], 
                                           df_toronto['Longitude'],
                                           df_toronto['Borough'], 
                                           df_toronto['Neighborhood']):
    label_text = borough + ' - ' + neighborhood
    label = folium.Popup(label_text)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=borough_color[borough],
        fill_color=borough_color[borough],
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

NameError: name 'lat_toronto' is not defined

In [38]:

CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [40]:
#Get venues for all neighborhoods in our dataset
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                latitudes=df_toronto['Latitude'],
                                longitudes=df_toronto['Longitude'])

NameError: name 'df_toronto' is not defined

In [41]:
#Check size of resulting dataframe
toronto_venues.shape

NameError: name 'toronto_venues' is not defined

In [42]:
toronto_venues.head()

NameError: name 'toronto_venues' is not defined

In [43]:
#Number of venues per neighborhood
toronto_venues.groupby('Neighborhood').count()

NameError: name 'toronto_venues' is not defined

In [44]:

#Number of unique venue categories
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

NameError: name 'toronto_venues' is not defined

In [45]:
#print out the list of categories
toronto_venues['Venue Category'].unique()[:100]

NameError: name 'toronto_venues' is not defined

In [46]:
# check if the results contain "Thai Restaurants"
#please note I changed the data to Thai because I was previously writing the code using Asian but the number is so small
"Thai Restaurant" in toronto_venues['Venue Category'].unique()

NameError: name 'toronto_venues' is not defined

In [47]:
# one hot encoding
to_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Neighborhoods'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head()

NameError: name 'toronto_venues' is not defined

In [48]:

to_grouped = to_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(to_grouped.shape)
to_grouped

NameError: name 'to_onehot' is not defined

In [49]:

len(to_grouped[to_grouped["Thai Restaurant"] > 0])

NameError: name 'to_grouped' is not defined

In [50]:
to_asian = to_grouped[["Neighborhoods","Thai Restaurant"]]

NameError: name 'to_grouped' is not defined

In [51]:
to_asian.head()

NameError: name 'to_asian' is not defined

In [52]:
# set number of clusters
toclusters = 3

to_clustering = to_asian.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=0).fit(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

NameError: name 'to_asian' is not defined

In [53]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
to_merged = to_asian.copy()

# add clustering labels
to_merged["Cluster Labels"] = kmeans.labels_

NameError: name 'to_asian' is not defined

In [54]:
to_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
to_merged.head()

NameError: name 'to_merged' is not defined

In [55]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
to_merged = to_merged.join(toronto_venues.set_index("Neighborhood"), on="Neighborhood")

print(to_merged.shape)
to_merged.head()

NameError: name 'to_merged' is not defined

In [56]:

# sort the results by Cluster Labels
print(to_merged.shape)
to_merged.sort_values(["Cluster Labels"], inplace=True)
to_merged

NameError: name 'to_merged' is not defined

In [57]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [58]:
# create map
map_clusters = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(toclusters)
ys = [i+x+(i*x)**2 for i in range(toclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_merged['Neighborhood Latitude'], to_merged['Neighborhood Longitude'], to_merged['Neighborhood'], to_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

NameError: name 'lat_toronto' is not defined

In [59]:
map_clusters.save('map_clusters.html')

NameError: name 'map_clusters' is not defined

In [60]:
to_merged.loc[to_merged['Cluster Labels'] == 0]

NameError: name 'to_merged' is not defined

In [61]:
#Cluster 1
to_merged.loc[to_merged['Cluster Labels'] == 1]

NameError: name 'to_merged' is not defined

In [62]:

#Cluster 2
to_merged.loc[to_merged['Cluster Labels'] == 2]

NameError: name 'to_merged' is not defined

# Observations
Most of Thai restaurants are in Cluster 2 which is around Adelaide, King, Richmond areas and lowest (close to zero) in Cluster 1 areas which are North Toronto West and Parkdale areas. Also, there are good opportunities to open near Chinatown, St James town as the competition seems to be low. Looking at nearby venues, it seems Cluster 1 might be a good location as there are not a lot of Asian restaurants in these areas. Therefore, this project recommends the entrepreneur to open an authentic Burmese restaurant in these locations with little to no competition. Nonetheless, if the food is authentic, affordable and good taste, I am confident that it will have great following everywhere